In [ ]:
import numpy as np
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split

from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, LearningRateScheduler
from keras.utils import to_categorical


In [ ]:
train_data = pd.read_csv('/content/image_train_Kaggle.csv')
test_data = pd.read_csv('/content/image_test_Kaggle.csv')


In [ ]:
Y = train_data.iloc[:,0]
X = train_data.iloc[:,1:]
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.1, random_state= 66)

In [ ]:
xtrain = xtrain.values.reshape(-1, 28, 28,1).astype("float")/255
xtest = xtest.values.reshape(-1, 28, 28,1).astype("float")/255


ytrain = to_categorical(ytrain)
ytest = to_categorical(ytest)

In [ ]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size=(3,3), activation='relu',
                 kernel_initializer = 'he_normal', strides=1,
                 padding='same',
                 input_shape = (28,28,1)))
model.add(BatchNormalization())

model.add(Conv2D(filters= 64, kernel_size=(3,3), activation = 'relu', strides = 1,
                 padding = 'same',
                 input_shape=(28,28,1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=(3,3), activation ='relu', strides = 1,
                 padding='same',
                 input_shape = (28,28,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters = 128, kernel_size=(3,3), activation='relu',
                 strides = 1, kernel_initializer = 'he_normal',
                 padding='same', data_format = 'channels_last',
                 input_shape = (28,28,1)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(filters=256, kernel_size=(3,3), activation='relu',
                 strides=1, padding='same', data_format='channels_last',
                 input_shape=(28,28,1)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(filters = 256, kernel_size=(3,3), activation='relu',
                 strides=1,kernel_initializer = 'he_normal',
                 padding = 'same', input_shape = (28,28,1) ))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))





model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.45))
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))


In [ ]:
optimizer= Adam(lr=0.0001, beta_1 = 0.99, beta_2 = 0.99)
#optimizer = SGD(lr = 0.001, momentum=0.99)

In [ ]:
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=["accuracy"])


In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor=0.2, patience = 3, min_lr = 0.0001)

reduce_lr = LearningRateScheduler(lambda x: 1e-3*0.9**x)

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.2,
    shear_range=0.2,
    width_shift_range = 0.1,
    height_shift_range=0.1,
    vertical_flip=True
    )


In [ ]:
#datagen.fit(xtrain)

In [ ]:
data_length = xtrain.shape[0]
model.fit_generator(datagen.flow(xtrain, ytrain, 
                                           batch_size=32),
                              epochs = 300,
                              validation_data = (xtest, ytest), verbose=2,
                              steps_per_epoch = data_length // 32,
                              callbacks = [reduce_lr])

In [ ]:
test_school = pd.read_csv("/content/image_test_Kaggle.csv")
ID = test_school['ID']
ID = ID.to_numpy()
test_school.drop('ID', axis = 1, inplace=True)
test_school = test_school.values.reshape((-1,28,28,1))
test_school = test_school.astype('float32')/255

In [ ]:

ypred = model.predict(test_school)
y_pred = []
for i in range(len(ypred)):
  y_pred += [np.argmax(ypred[i])]





In [ ]:
file = open('subfile_2.csv', 'w')
file.write('ID,label\n')
for i in range(len(y_pred)):
  file.write("{},{}\n".format(ID[i], y_pred[i]))
file.close